In [7]:
library(piggyback)
library(data.table)
library(dtplyr)
library(targets)
library(here)
library(stringdist)
library(tidyverse)
here::i_am("analysis/db_prep.ipynb");

here() starts at /dartfs-hpc/rc/home/k/f00345k/research/microbe_set_trait



This code is used to upload data onto GitHub using `piggyback` R package
```r
piggyback::pb_upload(file = here("large_files", "goldData.xlsx"), tag = "0.1", overwrite = TRUE)
```